In [ ]:
import sys
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from read_file import ReadFile
from schedule import CustomSchedule
from transformer import Transformer
from tokenizer import FrontmanTokenizer
from metrics import masked_loss, masked_accuracy

2025-05-19 15:19:24.927911: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747667965.123166      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747667965.185254      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

In [5]:
print(f'Tensorflow version: {tf.__version__}')
print(f'Python version: {sys.version}')

Tensorflow version: 2.18.0
Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


Tensorflow version: 2.17.1


Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]

In [ ]:
opus_file = 'data/train/opus_yo_en.tsv'        
raw_file = 'data/train/raw_yo_en.tsv'                
JW300 = 'data/train/JW300_en-yo.csv'                 
bible = 'data/train/bible_yo_en.csv'           
Government = 'data/train/Train.csv'                  


reader = ReadFile(dropna=True, drop_duplicates=True)

_, valid_df = reader.read_file([Government, opus_file, raw_file, bible, JW300],
                                     delimiter=',',
                                     shuffle=True, 
                                     split_ratio=0.1)

menyo_train = 'data/dev/train.tsv'
# education = 'data/dev/EnYo_Education.tsv'
monyo_dev = 'data/dev/dev.tsv'

menyo_test = 'data/test/test.tsv'


reader = ReadFile(dropna=True, drop_duplicates=True)


train_df = reader.read_file([menyo_train, monyo_dev],
                                     delimiter=',',
                                     shuffle=True, 
                                     split_ratio=None)

train_df = pd.concat([train_df, valid_df], axis=0, ignore_index=True)


valid_df = reader.read_file([menyo_test],
                                     delimiter=',',
                                     shuffle=False, 
                                     split_ratio=None)

train_df = reader.shuffle_df(train_df)

In [7]:
train_df.drop_duplicates(inplace=True, ignore_index=True)
train_df

,Yoruba,English
0,"Torí náà, nígbà tí Sámúẹ́lì rí i pé Jèhófà kò ...",So when Samuel learned that Jehovah had not ch...
1,Gbogbo ìgbà là ń gbàdúrà fún àwọn arákùnrin àt...,We pray for our brothers and sisters affected ...
2,4/<UNK>- Isere,4/<UNK>- Ishtar
3,Àfi ṣìí tí dírẹ́bà wa dúró lójijì!,Suddenly our driver stops!
4,"Bó sì ṣe yẹ́ kó rí náà nìyẹn, torí pé Jèhófà Ọ...","We would expect that to be true of the Bible, ..."
...,...,...
72548,"Ní ọdún méjì péré, ó lé ní ẹgbẹ̀rún méjìlá [ 1...","Over the next two years, more than 12,000 elde..."
72549,Mo rò wí pé ó ṣe pàtàkì.,I think it's quite significant.
72550,Ẹ jẹ́ ká gbé àpẹẹrẹ tá a tọ́ka sí lókè yìí yẹ̀...,"Let us take the example cited above, 1 Thessal..."
72551,Láti kọ́ si nípa lílo Kolibri láìsí lórí ìṣàso...,To learn more about using Kolibri in an offlin...


In [ ]:
tokenizer = FrontmanTokenizer(model_path='tokenizer/model/yo_en_bpe.model')

In [ ]:
def create_dataset(df, tokenizer, max_length=128, batch_size=128, drop_remainder=False, shuffle_size=False, cache=False):
    
    encoder_input = tokenizer.special_encode(
        df['English'].values.tolist(),      # Yoruba for Yoruba to English training 
        max_length=max_length,
        truncation=True,
        padding=True
    )
    
    decoder_input = tokenizer.special_encode(
        df['Yoruba'].values.tolist(),       # English for English to Yoruba training 
        max_length=max_length,
        truncation=True,
        padding=True,
        add_bos=True
    )

    decoder_target= tokenizer.special_encode(
        df['Yoruba'].values.tolist(),       # English for English to Yoruba training 
        max_length=max_length,
        truncation=True,
        padding=True,
        add_eos=True
    )
    
    dataset = tf.data.Dataset.from_tensor_slices(((
        encoder_input,
        decoder_input,
        ),
        decoder_target
        )
    )
    
    # dataset = tf.data.Dataset.from_tensor_slices((({
    #     "encoder_input_ids": encoder_input['input_ids'],
    #     "encoder_attention_mask": encoder_input['attention_mask'],
    #     "decoder_input_ids": decoder_input['input_ids'],
    #     "decoder_attention_mask": decoder_input['attention_mask'],}
    #     ),
    #     decoder_target
    #     )
    # )

    if cache:
        dataset = dataset.cache()
        
    if shuffle_size:
        dataset = dataset.shuffle(shuffle_size)
        
    if batch_size:
        dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)

    return dataset.prefetch(tf.data.AUTOTUNE)

In [10]:
train_set = create_dataset(
    df=train_df,
    tokenizer=tokenizer,
    shuffle_size=100_000,
    cache=True
)

valid_set = create_dataset(
    df=valid_df,
    tokenizer=tokenizer,
    cache=True
)

train_set

I0000 00:00:1747667989.296459      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None)), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None))>

In [ ]:
model_path ='models/pretrained/EnYo BaseModel.keras'

custom_objects={
    'masked_accuracy':masked_accuracy,
    'masked_loss':masked_loss
}

model = tf.keras.models.load_model(model_path, custom_objects=custom_objects)

weight_path = 'models/weights/weight_epoch_08.weights.h5' 

model.load_weights(weight_path)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'sequential' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'sequential_1' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'sequential_2' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.1

In [12]:
optimizer = tf.keras.optimizers.Adam(0.00001)


model.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [13]:
# debug_dataset = train_set.unbatch().take(512).batch(128)
# debug_dataset

In [ ]:
best_model_cbk = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/finetuned/EnYo FineTunedBest.keras',
    save_weights_only=False,  
    monitor='val_masked_accuracy',       
    mode='max',              
    save_best_only=True, 
    verbose=1
)

early_stopping_cbk = tf.keras.callbacks.EarlyStopping(
    monitor='val_masked_accuracy',
    patience=5,              
    min_delta = 0.005,
    verbose=1,
    mode='max',              
    restore_best_weights=True,
    start_from_epoch=1
)

In [15]:
# # freeze the first N decoder layers

# # N = 2

# for layer in model.layers[1].decoder_layers:
#     layer.trainable = True
    

# # freeze the first all encoder layers

# for layer in model.layers[0].encoder_layers:
#     layer.trainable = True

In [16]:

model.dropout_rate = 0.1

In [ ]:
model.fit(train_set, epochs=20,
                    validation_data=valid_set,
                    callbacks=[best_model_cbk, early_stopping_cbk]
                   )

model.save("models/pretrained/EnYo FineTunedBase.keras")

Epoch 1/20


I0000 00:00:1747668050.448863      72 service.cc:148] XLA service 0x7f1ca0003e60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747668050.449756      72 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1747668052.732265      72 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1747668054.145043      72 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747668074.135930      72 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


566/567 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - loss: 2.1523 - masked_accuracy: 0.5809

W0000 00:00:1747668258.826606      73 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step - loss: 2.1523 - masked_accuracy: 0.5809

W0000 00:00:1747668282.109580      70 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1747668288.621831      72 assert_op.cc:38] Ignoring Assert operator compile_loss/masked_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert



Epoch 1: val_masked_accuracy improved from -inf to 0.38637, saving model to best_model.keras
567/567 ━━━━━━━━━━━━━━━━━━━━ 281s 382ms/step - loss: 2.1522 - masked_accuracy: 0.5809 - val_loss: 3.8307 - val_masked_accuracy: 0.3864
Epoch 2/20
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - loss: 2.1196 - masked_accuracy: 0.5852
Epoch 2: val_masked_accuracy improved from 0.38637 to 0.39261, saving model to best_model.keras
567/567 ━━━━━━━━━━━━━━━━━━━━ 190s 334ms/step - loss: 2.1195 - masked_accuracy: 0.5852 - val_loss: 3.7644 - val_masked_accuracy: 0.3926
Epoch 3/20
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - loss: 2.0893 - masked_accuracy: 0.5886
Epoch 3: val_masked_accuracy improved from 0.39261 to 0.39739, saving model to best_model.keras
567/567 ━━━━━━━━━━━━━━━━━━━━ 189s 334ms/step - loss: 2.0893 - masked_accuracy: 0.5886 - val_loss: 3.7200 - val_masked_accuracy: 0.3974
Epoch 4/20
567/567 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - loss: 2.0515 - masked_accuracy: 0.5938
Epoch 4: val_masked_accu